In [1]:
from api_token import token
import todoist as td
import pandas as pd
import datetime as dt

In [2]:
# Get yesterday's date to feed into call
today = dt.date.today()
yesterday = today - dt.timedelta(days = 1)
yday_start = str(yesterday)+"T00:00:00Z"
yday_end = str(yesterday)+"T23:59:59Z"
print(yday_start)
print(yday_end)

2021-01-05T00:00:00Z
2021-01-05T23:59:59Z


In [3]:
api = td.TodoistAPI(token)
# Sync to API to get latest data
api.sync();

In [84]:
yday_completed_tasks_dict = api.completed.get_all(since = yday_start, until = yday_end)

In [85]:
# get list of keys
[key for key in yday_completed_tasks_dict]

['items', 'projects']

In [86]:
yday_completed_tasks_df = pd.DataFrame(yday_completed_tasks_dict['items'])

In [87]:
yday_completed_tasks_df.head()

,completed_date,content,id,meta_data,project_id,task_id,user_id
0,2021-01-05T15:40:59Z,**03a** BUS-4197 [(DE-7570)](https://justeatta...,4177667603,None,2240338162,4430222040,28966387
1,2021-01-05T15:40:56Z,**03b** Re-read latest slack message from Fred...,4177667478,None,2240338162,4431395486,28966387
2,2021-01-05T13:42:33Z,Complete WFH training Fwd: Help with Temporary...,4177354628,None,2240338162,4430173573,28966387
3,2021-01-05T12:50:20Z,**01** Make notes on Data Engineer JD's,4177236599,None,2240338076,4435958173,28966387
4,2021-01-05T10:59:35Z,**E1** Call Quality Surveyors on 020 8185 7454...,4177022640,None,2240338076,4436808693,28966387


In [88]:
# our df doesn't yet have project name
# create new dict of id and project name
td_projects_dict = yday_completed_tasks_dict['projects']
projects = {int(key):td_projects_dict[key]['name'] for key in td_projects_dict}
projects

{2240338076: '❶ This Week', 2240338162: '🥓 JET', 2240717407: '⏱ Routines'}

In [89]:
# map project names to df
yday_completed_tasks_df['project_name'] = yday_completed_tasks_df['project_id'].map(projects)

In [90]:
# TODO to get task url you have to create it as follows https://todoist.com/showTask?id={task_id}
yday_completed_tasks_df['task_url'] = yday_completed_tasks_df['task_id'].map('https://todoist.com/showTask?id={}'.format)

In [91]:
# Drop unwanted columns
yday_completed_tasks_df = yday_completed_tasks_df.drop(columns = ['id', 'meta_data', 'project_id', 'user_id', 'task_id'])

In [92]:
yday_completed_tasks_df

,completed_date,content,project_name,task_url
0,2021-01-05T15:40:59Z,**03a** BUS-4197 [(DE-7570)](https://justeatta...,🥓 JET,https://todoist.com/showTask?id=4430222040
1,2021-01-05T15:40:56Z,**03b** Re-read latest slack message from Fred...,🥓 JET,https://todoist.com/showTask?id=4431395486
2,2021-01-05T13:42:33Z,Complete WFH training Fwd: Help with Temporary...,🥓 JET,https://todoist.com/showTask?id=4430173573
3,2021-01-05T12:50:20Z,**01** Make notes on Data Engineer JD's,❶ This Week,https://todoist.com/showTask?id=4435958173
4,2021-01-05T10:59:35Z,**E1** Call Quality Surveyors on 020 8185 7454...,❶ This Week,https://todoist.com/showTask?id=4436808693
5,2021-01-05T10:59:35Z,Send Solicitor details to George,❶ This Week,https://todoist.com/showTask?id=4463146803
6,2021-01-05T10:59:35Z,Send solicitor details to Daniel at Churchill,❶ This Week,https://todoist.com/showTask?id=4463147630
7,2021-01-05T10:23:24Z,**00** Plan my day,❶ This Week,https://todoist.com/showTask?id=4355453459
8,2021-01-05T09:37:54Z,**E1** Email George to ask about progress with...,❶ This Week,https://todoist.com/showTask?id=4459378021
9,2021-01-05T08:24:33Z,Oranges,⏱ Routines,https://todoist.com/showTask?id=4462624101


In [ ]:
# send email

import yagmail

receiver = "your@gmail.com"
body = "Hello there from Yagmail"
filename = "document.pdf"

yag = yagmail.SMTP("my@gmail.com")
yag.send(
    to=receiver,
    subject="Yagmail test with attachment",
    contents=body, 
    attachments=filename,
)

In [97]:
# from send_email import send_mail
# from get_data import get_gdp_data
from pretty_html_table import build_table


def prettify_data(data):
#     gdp_data = get_gdp_data()
    output = build_table(data, 'blue_light')
#     send_mail(output)
    return output


# prettify_data(yday_completed_tasks_df)

In [101]:
import yagmail
yag = yagmail.SMTP("nibe0993")
contents = [
    "Testing \n", prettify_data(yday_completed_tasks_df) ]
yag.send('nibe12@live.com', 'subject', contents)

Password for <nibe0993@gmail.com>: ········
Save username and password in keyring? [y/n]: n


SMTPAuthenticationError: (535, b'5.7.8 Username and Password not accepted. Learn more at\n5.7.8  https://support.google.com/mail/?p=BadCredentials a13sm4378244wrt.96 - gsmtp')

In [29]:
# Order of columns ["task_content_description", "project_name", "completed_at"
# , "labels", "priority", "task_link_url", "due_date"]

# TODOs
# add column with label names
# add column with project name
# add column with url link